### Handling missing values
In this Jupyter notebook I will be looking at missing values, outliers and how I am going to handle these.
<br>
<br>
First of I need to calculate how many missing items there are in each row, however the file is too big to load into memory in a single file. This is where dask comes in; dask is a library that handles splitting up files and loading them into memory one at a time and calculating the results. Using dask I can set a maximum size that is allowed to be loaded into memory at ones, 32MB should be fine and still reasonably fast.
<br>
Below you can see the results of columns with their missing values total and percentage.

| Column                          | Missing Count | Missing Percent|
|---------------------------------|---------------|----------------|
| Service:RDT-ID                  | 0             | 0.000000       |
| Service:Date                    | 0             | 0.000000       |
| Service:Type                    | 0             | 0.000000       |
| Service:Company                 | 0             | 0.000000       |
| Service:Train number            | 0             | 0.000000       |
| Service:Completely cancelled    | 0             | 0.000000       |
| Service:Partly cancelled        | 0             | 0.000000       |
| Service:Maximum delay           | 0             | 0.000000       |
| Stop:RDT-ID                     | 0             | 0.000000       |
| Stop:Station code               | 182,170       | 0.125885       |
| Stop:Station name               | 0             | 0.000000       |
| Stop:Arrival time               | 16,127,855    | 11.144837      |
| Stop:Arrival delay              | 16,127,855    | 11.144837      |
| Stop:Arrival cancelled          | 16,127,855    | 11.144837      |
| Stop:Departure time             | 15,859,427    | 10.959345      |
| Stop:Departure delay            | 15,859,427    | 10.959345      |
| Stop:Departure cancelled        | 15,859,427    | 10.959345      |
| Stop:Platform change            | 0             | 0.000000       |
| Stop:Planned platform           | 16,970,704    | 11.727271      |
| Stop:Actual platform            | 16,970,704    | 11.727271      |

<br><br>


In [2]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd

df = pd.read_csv('NS_Data\combined_trein_data.csv', nrows=1_000_000)
df.to_csv('NS_Data\combined_trein_data_sample.csv', index=False)

<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\bramm\AppData\Local\Temp\ipykernel_23756\1267378841.py:5: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('NS_Data\combined_trein_data.csv', nrows=1_000_000)
C:\Users\bramm\AppData\Local\Temp\ipykernel_23756\1267378841.py:6: SyntaxWarning: invalid escape sequence '\c'
  df.to_csv('NS_Data\combined_trein_data_sample.csv', index=False)


In [4]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd

def missing_value_summary(csv_path, blocksize="64MB"):
    
    """
    Compute missing value counts and percentages for a CSV file using Dask.

    Parameters:
        csv_path (str): Path to the CSV file.
        blocksize (str or int): Block size for Dask to load the CSV in chunks. Default is "32MB".

    Returns:
        pd.DataFrame: DataFrame containing missing value counts and percentages per column.
    """

    # Load CSV lazily with Dask
    df = dd.read_csv(csv_path, blocksize=blocksize)

    with ProgressBar():
        missing_summary = df.isnull().sum().compute()
        row_count = df.shape[0].compute()

    missing_percent = (missing_summary / row_count) * 100

    return pd.DataFrame({
        "missing_count": missing_summary,
        "missing_percent": missing_percent
    })

# Example usage
if __name__ == "__main__":
    summary = missing_value_summary("NS_Data\combined_trein_data_sample.csv", blocksize="64MB")
    print(summary)


<>:34: SyntaxWarning: invalid escape sequence '\c'
<>:34: SyntaxWarning: invalid escape sequence '\c'
C:\Users\bramm\AppData\Local\Temp\ipykernel_23756\80343318.py:34: SyntaxWarning: invalid escape sequence '\c'
  summary = missing_value_summary("NS_Data\combined_trein_data_sample.csv", blocksize="64MB")


ImportError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: pyarrow>=10.0.1 is required for PyArrow backed StringArray.

In [2]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

def replace_missing_with_none(csv_path, column_name, blocksize="64MB", output_path=None):
    """
    Replace missing values (NaN and empty strings) in a specified column with "NOCODE".

    Parameters:
        csv_path (str): Path to the CSV file.
        column_name (str): Column to replace missing values in.
        blocksize (str or int): Block size for Dask to load the CSV in chunks.
        output_path (str, optional): Path to save the modified CSV. If None, returns the Dask DataFrame.

    Returns:
        dask.dataframe.DataFrame or None: Modified DataFrame if output_path is None, otherwise saves to file.
    """
    # Load CSV lazily
    df = dd.read_csv(csv_path, blocksize=blocksize)

    # Replace empty strings and NaN values with "NOCODE"
    df[column_name] = df[column_name].replace("", "NOCODE").fillna("NOCODE")

    if output_path:
        with ProgressBar():
            df.to_csv(output_path, single_file=True, index=False)
        print(f"Modified dataset saved to {output_path}")
        return None
    else:
        return df


def replace_missing_with_value(csv_path, columns, value, blocksize="64MB", output_path=None):
    """
    Replace missing values (NaN and empty strings) in specified columns with a given value.

    Parameters:
        csv_path (str): Path to the CSV file.
        columns (str or list): Column name(s) to replace missing values in.
        value: The value to insert for missing values.
        blocksize (str or int): Block size for Dask to load the CSV in chunks.
        output_path (str, optional): Path to save the modified CSV. If None, returns the Dask DataFrame.

    Returns:
        dask.dataframe.DataFrame or None: Modified DataFrame if output_path is None, otherwise saves to file.
    """
    # Ensure columns is a list
    if isinstance(columns, str):
        columns = [columns]

    # Load CSV lazily
    df = dd.read_csv(csv_path, blocksize=blocksize)

    # Replace missing values in each specified column
    for col in columns:
        df[col] = df[col].replace("", value).fillna(value)

    if output_path:
        with ProgressBar():
            df.to_csv(output_path, single_file=True, index=False)
        print(f"Modified dataset saved to {output_path}")
        return None
    else:
        return df

In [6]:

replace_missing_with_none(
        "NS_Data\combined_trein_data_sample.csv",
        "Stop:Station code",
        blocksize="32MB",
        output_path="NS_Data/combined_trein_data_modified.csv"
)
replace_missing_with_value(
        "NS_Data/combined_trein_data_modified.csv",
        ["Stop:Planned platform", "Stop:Actual platform"],
        1,
        blocksize="32MB",
        output_path="NS_Data/combined_trein_data_modified.csv"
)
replace_missing_with_value(
    "NS_Data/combined_trein_data_modified.csv",
    ["Stop:Arrival time", "Stop:Departure time"],
    -1,
    blocksize="32MB",
    output_path="NS_Data/combined_trein_data_modified.csv"
)
replace_missing_with_value(
    "NS_Data/combined_trein_data_modified.csv",
    ["Stop:Arrival delay", "Stop:Departure delay"],
    0,
    blocksize="32MB",
    output_path="NS_Data/combined_trein_data_modified.csv"
)
replace_missing_with_value(
    "NS_Data/combined_trein_data_modified.csv",
    ["Stop:Arrival cancelled", "Stop:Departure cancelled"],
    False,
    blocksize="32MB",
    output_path="NS_Data/combined_trein_data_modified.csv"
)
missing_value_summary("NS_Data/combined_trein_data_modified.csv", blocksize="64MB")

[                                        ] | 0% Completed | 102.82 ms

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\bramm\AppData\Local\Temp\ipykernel_21224\488088290.py:2: SyntaxWarning: invalid escape sequence '\c'
  "NS_Data\combined_trein_data_sample.csv",


[########################################] | 100% Completed | 8.68 ss
Modified dataset saved to NS_Data/combined_trein_data_modified.csv
[########################################] | 100% Completed | 7.53 ss
Modified dataset saved to NS_Data/combined_trein_data_modified.csv
[########################################] | 100% Completed | 7.77 ss
Modified dataset saved to NS_Data/combined_trein_data_modified.csv
[########################################] | 100% Completed | 7.42 ss
Modified dataset saved to NS_Data/combined_trein_data_modified.csv
[########################################] | 100% Completed | 7.79 ss
Modified dataset saved to NS_Data/combined_trein_data_modified.csv
[########################################] | 100% Completed | 2.60 ss
[########################################] | 100% Completed | 1.48 ss


,missing_count,missing_percent
Service:RDT-ID,0,0.0
Service:Date,0,0.0
Service:Type,0,0.0
Service:Company,0,0.0
Service:Train number,0,0.0
Service:Completely cancelled,0,0.0
Service:Partly cancelled,0,0.0
Service:Maximum delay,0,0.0
Stop:RDT-ID,0,0.0
Stop:Station code,0,0.0
